# Loss functions

**Goal:** Build up some intuition for the Hungarian loss, and how to build it in python :) 

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import datetime
import time

import collections
import tensorflow as tf
import tensorflow_datasets as tfds
from absl import logging

tf.random.set_seed(0)

# The slot attn code
import os
os.sys.path.append('../google-research')

import slot_attention.data as data_utils
import slot_attention.model as model_utils
import slot_attention.utils as utils

from slot_attention.set_prediction.train import train_step

%load_ext autoreload
%autoreload 2

In [2]:
batch_size = 512
num_slots = 3
nPhotons = 2
nFeatures = 3 # (x,y,E)

y_pred = tf.random.normal((batch_size, num_slots, nFeatures))
y_true = tf.random.normal((batch_size, nPhotons,  nFeatures))

In [3]:
def hungarian_weighted_MSE_loss(x, y):
    """Huber loss for sets, matching elements with the Hungarian algorithm.

    This loss is used as reconstruction loss in the paper 'Deep Set Prediction
    Networks' https://arxiv.org/abs/1906.06565, see Eq. 2. For each element in the
    batches we wish to compute min_{pi} ||y_i - x_{pi(i)}||^2 where pi is a
    permutation of the set elements. We first compute the pairwise distances
    between each point in both sets and then match the elements using the scipy
    implementation of the Hungarian algorithm. This is applied for every set in
    the two batches. Note that if the number of points does not match, some of the
    elements will not be matched. As distance function we use the Huber loss.

    Args:
    x: Batch of sets of size [batch_size, n_points, dim_points]. Each set in the
      batch contains n_points many points, each represented as a vector of
      dimension dim_points.
    y: Batch of sets of size [batch_size, n_points, dim_points].

    Returns:
    Average distance between all sets in the two batches.
    """
    pairwise_cost = tf.losses.Huber(reduction=tf.keras.losses.Reduction.NONE)(
      tf.expand_dims(y, axis=-2), tf.expand_dims(x, axis=-3))
    indices = np.array(
      list(map(scipy.optimize.linear_sum_assignment, pairwise_cost)))

    transposed_indices = np.transpose(indices, axes=(0, 2, 1))

    actual_costs = tf.gather_nd(
      pairwise_cost, transposed_indices, batch_dims=1)

    return tf.reduce_mean(tf.reduce_sum(actual_costs, axis=1))

In [4]:
tf.expand_dims(y_true, axis=-2).shape

TensorShape([512, 2, 1, 3])

In [5]:
tf.expand_dims(y_pred, axis=-3).shape

TensorShape([512, 1, 3, 3])

In [6]:
mse = (tf.expand_dims(y_true, axis=-2) - tf.expand_dims(y_pred, axis=-3))**2

# Sum over the final features
MSE_manual = tf.math.reduce_mean(mse,axis=-1)
MSE_manual.shape

TensorShape([512, 2, 3])

In [7]:
MSE_tf = tf.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)(
    tf.expand_dims(y_true, axis=-2), tf.expand_dims(y_pred, axis=-3))
MSE_tf.shape

TensorShape([512, 2, 3])

In [8]:
tf.reduce_max(MSE_manual - MSE_tf)

<tf.Tensor: shape=(), dtype=float32, numpy=1.9073486e-06>

OK, I understand what this function is doing (up to floating point errors!!

In [9]:
Huber_tf = tf.losses.Huber(reduction=tf.keras.losses.Reduction.NONE)(
    tf.expand_dims(y_true, axis=-2), tf.expand_dims(y_pred, axis=-3))
Huber_tf.shape

TensorShape([512, 2, 3])

In [10]:
tf.losses.Huber

keras.losses.Huber

In [11]:
from scipy.optimize import linear_sum_assignment

`linear_sum_assignment` solves the problem of minimizing a cost matrix

In [12]:
?linear_sum_assignment

Signature: linear_sum_assignment(cost_matrix, maximize=False)
Docstring:
Solve the linear sum assignment problem.

The linear sum assignment problem is also known as minimum weight matching
in bipartite graphs. A problem instance is described by a matrix C, where
each C[i,j] is the cost of matching vertex i of the first partite set
(a "worker") and vertex j of the second set (a "job"). The goal is to find
a complete assignment of workers to jobs of minimal cost.

Formally, let X be a boolean matrix where :math:`X[i,j] = 1` iff row i is
assigned to column j. Then the optimal assignment has cost

.. math::
    \min \sum_i \sum_j C_{i,j} X_{i,j}

where, in the case where the matrix X is square, each row is assigned to
exactly one column, and each column to exactly one row.

This function can also solve a generalization of the classic assignment
problem where the cost matrix is rectangular. If it has more rows than
columns, then not every row needs to be assigned to a column, and vice
vers

In [22]:
?linear_sum_assignment

Signature: linear_sum_assignment(cost_matrix, maximize=False)
Docstring:
Solve the linear sum assignment problem.

The linear sum assignment problem is also known as minimum weight matching
in bipartite graphs. A problem instance is described by a matrix C, where
each C[i,j] is the cost of matching vertex i of the first partite set
(a "worker") and vertex j of the second set (a "job"). The goal is to find
a complete assignment of workers to jobs of minimal cost.

Formally, let X be a boolean matrix where :math:`X[i,j] = 1` iff row i is
assigned to column j. Then the optimal assignment has cost

.. math::
    \min \sum_i \sum_j C_{i,j} X_{i,j}

where, in the case where the matrix X is square, each row is assigned to
exactly one column, and each column to exactly one row.

This function can also solve a generalization of the classic assignment
problem where the cost matrix is rectangular. If it has more rows than
columns, then not every row needs to be assigned to a column, and vice
vers

In [13]:
idx_rows, idx_cols = linear_sum_assignment(MSE_tf[0])
idx_rows, idx_cols

(array([0, 1]), array([0, 2]))

**Output:** Array of row indices and column indices corresponding to the optimal assignment

In [14]:
MSE_tf[0]

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.08726764, 2.491103  , 1.1471361 ],
       [0.87472844, 1.4417597 , 1.3980832 ]], dtype=float32)>

OK, based on looking at this, I think was `linear_sum_assignment` returns the row and column indices for the adjacency matrix that solves this bipartite graph matching problem.

In [15]:
# Let X be shape (2,3) be the adjacency matrix
X = np.zeros_like(MSE_tf[0])
X[idx_rows, idx_cols] = 1

In [16]:
X

array([[1., 0., 0.],
       [0., 0., 1.]], dtype=float32)

In [17]:
np.sum(X * MSE_tf[0].numpy())

1.4853508

In [18]:
indices = np.array( list(map(linear_sum_assignment, MSE_tf)) )

In [19]:
indices.shape

(512, 2, 2)

In [20]:
indices[0]

array([[0, 1],
       [0, 2]])

(I think) the output becomes a bit more transparent when we take the transpose (which is in fact what they're doing in the loss function calculation.

In [23]:
transposed_indices = np.transpose(indices, axes=(0, 2, 1))

In [24]:
transposed_indices[0]

array([[0, 0],
       [1, 2]])

**What does this mean?**
- Worker (truth particle) 0 matches to job (slot) 0
- Worker (truth particle) 1 matches to job (slot) 2


In [21]:
?tf.gather_nd

Signature: tf.gather_nd(params, indices, batch_dims=0, name=None)
Docstring:
Gather slices from `params` into a Tensor with shape specified by `indices`.

`indices` is an K-dimensional integer tensor, best thought of as a
(K-1)-dimensional tensor of indices into `params`, where each element defines
a slice of `params`:

    output[\\(i_0, ..., i_{K-2}\\)] = params[indices[\\(i_0, ..., i_{K-2}\\)]]

Whereas in `tf.gather` `indices` defines slices into the first
dimension of `params`, in `tf.gather_nd`, `indices` defines slices into the
first `N` dimensions of `params`, where `N = indices.shape[-1]`.

The last dimension of `indices` can be at most the rank of
`params`:

    indices.shape[-1] <= params.rank

The last dimension of `indices` corresponds to elements
(if `indices.shape[-1] == params.rank`) or slices
(if `indices.shape[-1] < params.rank`) along dimension `indices.shape[-1]`
of `params`.  The output tensor has shape

    indices.shape[:-1] + params.shape[indices.shape[-1]:]

Ad